In [5]:
import numpy as np
import random

In [6]:
class AntColonyOptimizer:
    def __init__(self, connections, n_ants, n_cities, alpha, beta, evaporation_rate, max_time, city_names, cost_weight):
        self.connections = connections
        self.n_ants = n_ants
        self.n_cities = n_cities
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.max_time = max_time
        self.city_names = city_names
        self.cost_weight = cost_weight
        self.times, self.costs = self.calculate_times_and_costs()

    def calculate_times_and_costs(self):
        times = np.zeros((self.n_cities, self.n_cities))
        costs = np.zeros((self.n_cities, self.n_cities))
        for conn in self.connections:
            times[conn[0], conn[1]] = conn[2]
            times[conn[1], conn[0]] = conn[2]
            costs[conn[0], conn[1]] = conn[3]
            costs[conn[1], conn[0]] = conn[3]
        return times, costs

    def run(self, n_iterations):
      pheromone_matrix = np.ones((self.n_cities, self.n_cities))
      visibility = self.calculate_visibility()
    
      best_path = None
      best_time = float('inf')
      best_cost = float('inf')

      for _ in range(n_iterations):
          all_paths = []
          all_times = []
          all_costs = []
          for _ in range(self.n_ants):
              path, time, cost = self.generate_path(pheromone_matrix)
              all_paths.append(path)
              all_times.append(time)
              all_costs.append(cost)

          pheromone_matrix = self.update_pheromones(pheromone_matrix, all_paths, all_times, all_costs)

          min_index = np.argmin(all_times)
          min_time = all_times[min_index]
          min_cost = all_costs[min_index]
          if min_time < best_time:
              best_path = all_paths[min_index]
              best_time = min_time
              best_cost = min_cost

          if best_time <= self.max_time:
              break

      return best_path, best_time, best_cost


    def generate_solutions(self, pheromone_matrix):
        all_paths = []
        all_costs = []

        for _ in range(self.n_ants):
            path, cost = self.generate_path(pheromone_matrix)
            all_paths.append(path)
            all_costs.append(cost)

        return all_paths, all_costs

    def generate_path(self, pheromone_matrix):
      visited = [np.random.randint(self.n_cities)]
      moves_limit = 17

      for _ in range(moves_limit - 1):
          next_city = self.pick_next_city(visited[-1], visited, pheromone_matrix)
          visited.append(next_city)
        
          # Check if all cities have been visited
          if len(set(visited)) == self.n_cities:
              break

      total_time = sum(self.times[visited[i], visited[i + 1]] for i in range(len(visited) - 1)) + self.times[visited[-1], visited[0]]
      total_cost = sum(self.costs[visited[i], visited[i + 1]] for i in range(len(visited) - 1)) + self.costs[visited[-1], visited[0]]
    
      return visited, total_time, total_cost

    def calculate_visibility(self):
      visibility = np.zeros((self.n_cities, self.n_cities))
      for i in range(self.n_cities):
          for j in range(self.n_cities):
              if i != j:
                  visibility[i, j] = 1 / self.times[i, j]
      return visibility


    def pick_next_city(self, current_city, visited, pheromone_matrix):
      probabilities = []
      unvisited_cities_weight = 650  # Increase this value to prioritize visiting unvisited cities

      for city in range(self.n_cities):
          if self.times[current_city, city] == 0:
              probabilities.append(0)
          else:
              city_weight = unvisited_cities_weight if city not in visited else 1
              probabilities.append(
                  (pheromone_matrix[current_city, city] ** self.alpha) * ((1 / self.times[current_city, city]) ** self.beta) * city_weight
              )

      probabilities_sum = sum(probabilities)

      if probabilities_sum == 0:
          # If probabilities_sum is zero, choose the next city uniformly at random
          return np.random.randint(self.n_cities)
      else:
          probabilities = [prob / probabilities_sum for prob in probabilities]
          return np.random.choice(self.n_cities, p=probabilities)


    def update_pheromones(self, pheromone_matrix, all_paths, all_times, all_costs):
        pheromone_matrix *= (1 - self.evaporation_rate)

        for path, time, cost in zip(all_paths, all_times, all_costs):
            weighted_value = time + self.cost_weight * cost
            for i in range(len(path) - 1):
                pheromone_matrix[path[i], path[i+1]] += 1 / weighted_value

        return pheromone_matrix

In [7]:
def main():
    connections = [
         (0, 1, 136 / 60, 98),
         (1, 2, 82 / 60, 80),
         (1, 6, 480 / 60, 345),
         (1, 9, 112 / 60, 185),
         (1, 11, 225 / 60, 380),
         (1, 10, 390 / 60, 400),
         (2, 3, 105 / 60, 48),
         (3, 4, 120 / 60, 40),
         (3, 5, 364 / 60, 235),
         (4, 6, 120 / 60, 40),
         (5, 6, 232 / 60, 125),
         (6, 7, 464 / 60, 240),
         (7, 8, 168 / 60, 125),
         (7, 9, 174 / 60, 180),
         (9, 10, 200 / 60, 320),
         (10, 11, 150 / 60, 98),
    ]

    city_names = [
        "London", "Paris", "Brussels", "Amsterdam", "Cologne",
        "Berlin", "Frankfurt", "Milan", "Rome", "Lyon",
        "Barcelona", "Madrid"
    ]

    n_cities = len(city_names)
    aco = AntColonyOptimizer(
      connections=connections,
      n_ants=20,
      n_cities=n_cities,
      alpha=1,
      beta=5,
      evaporation_rate=0.1,
      max_time=72,
      city_names=city_names,
      cost_weight=0.5  # Change this value to adjust the trade-off
    )

    best_path, best_time, best_cost = aco.run(n_iterations=220)

    if best_time > 72:
        raise Exception("No solution found within the 72-hour limit.")
    else:
        best_path_names = [city_names[i] for i in best_path]
        print("Best path found:", best_path_names)
        print("Time:", best_time, "hours")
        print("Cost:", best_cost)

In [26]:
main()

Best path found: ['Berlin', 'Frankfurt', 'Cologne', 'Amsterdam', 'Brussels', 'Paris', 'London', 'Paris', 'Lyon', 'Milan', 'Rome', 'Milan', 'Lyon', 'Barcelona', 'Madrid']
Time: 34.61666666666667 hours
Cost: 1742.0


/var/folders/s4/_2snl14x39d8z8sl49qsphn40000gp/T/ipykernel_16410/2833705254.py:91: RuntimeWarning: divide by zero encountered in double_scalars
  visibility[i, j] = 1 / self.times[i, j]
